# Tutorial 3: LQR for a double pendulum
In this tutorial we will use our knowledge about the double pendulum to create optimal controllers that hold keep the robot in one of its unstable points of equilibrium. We will also introduce the acrobot and pendubot, two classical examples of underactuated robots. 

**Pre-requisites**

Knowledge of the dynamics of the double pendulum, as well as basic notions of LQRs and the concept of the region of attraction.

**Goals**

Deriving and implementing a linear quadratic regulators for an underactuated double pendulum.

This notebook is organized as follows:

    1. Derivation
    2. Implementation
    3. Region of attraction
    4. Maximizing the RoA with CMA-ES

Run the next cell to make sure you have all the necessary dependencies installed.

In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
import numpy as np
import pandas as pd
import sympy as smp
from datetime import datetime
from double_pendulum.model.symbolic_plant import SymbolicDoublePendulum, diff_to_matrix, sub_symbols
from double_pendulum.model.model_parameters import model_parameters
from double_pendulum.simulation.simulation import Simulator
from double_pendulum.controller.lqr.lqr_controller import LQRController
from double_pendulum.utils.plotting import plot_timeseries, plotEllipseSections
from double_pendulum.utils.misc_roa import sampleFromEllipsoid, quadForm
from double_pendulum.filter.lowpass import lowpass_filter
#from double_pendulum.utils.plotting import direct_sphere, sample_from_ellipsoid
from double_pendulum.model.plant import DoublePendulumPlant
from double_pendulum.controller.lqr.roa.roa_paropt import roa_lqr_opt


%matplotlib inline
from IPython.display import HTML, display
import matplotlib as mpl
mpl.rcParams['animation.writer'] = "pillow"
import matplotlib.pyplot as plt

The next cell introduces the model of the robot. You can change the between acrobot and pendubot by commenting/uncommenting the second and third lines. In this cell we also generate the plant and simulator. These allow us to run simulations as well as experiments on the real system. Update the first line of the next cell with the motors on the pendulum assigned to you in order to be able to run experiments.

In [ ]:
motors = [403, 379] # REPLACE WITH YOUR MOTOR IDS [SHOULDER, ELBOW]
model = "model_07"
#robot = "acrobot"
robot = "pendubot"

if robot == "pendubot":
    torque_limit = [0.15, 0.0]
    active_act = 0
elif robot == "acrobot":
    torque_limit = [0.0, 0.15]
    active_act = 1
friction_compensation=False
model_par_path = "identified_parameters/" + model + "/model_parameters.yml"
mpar = model_parameters(filepath=model_par_path)
mpar_con = model_parameters(filepath=model_par_path)
mpar_con.set_motor_inertia(0.0)
mpar_con.set_damping([0.0, 0.0])
mpar_con.set_cfric([0.0, 0.0])
if friction_compensation:
    mpar_con.set_damping([0.004,0.004])
    mpar_con.set_cfric([0.00305,0.0007777])
mpar_con.set_torque_limit(torque_limit)
# mpar.set_length([0.05,0.10])
print(mpar_con)

dt = 0.002
tf = 10
integrator = "runge_kutta"

plant = SymbolicDoublePendulum(model_pars=mpar_con)
sim = Simulator(plant=plant)

## 1. Derivation

The following section is based on Chapter 8 of Russ Tedrake's Underactuated Robotics.

The Linear-Quadratic Regulator (LQR) is the simplest example of an optimal controller. It provides a controller that, subjet to a linear-quadratic cost function, optimizes the evolution of a linear system. This means that need to formulate the dynamics of the system so that they conform to the following shape:

$$
\dot{\mathbf{x}} = f(\mathbf{x}, \mathbf{u}) = A \mathbf{x} + B \mathbf{u}
$$

We can then define the linear-quadratic cost function:

$$
J = \int_{t_0}^{t_f} \left(\mathbf{x}^T Q \mathbf{x} + \mathbf{u}^T R \mathbf{u} \right)dt
$$

where Q and R are symmetric positive-definite matrices.

### The Hamilton-Jacobi-Bellman equation

To solve the infinite-horizon optimal control problem, we use the Hamilton-Jacobi-Bellman equation.

$$
\min_{\mathbf{u}} \left( \ell(\mathbf{x}, \mathbf{u}) + \frac{\partial J^*}{\partial \mathbf{x}} f(\mathbf{x}, \mathbf{u}) \right) = 0
$$

where $\ell$ is the cost function. and $f$ is the system dynamics (in the shape $\dot{\mathbf{x}} = f(\mathbf{x}, \mathbf{u})$). $J^*$ is the optimal cost-to-go, that is, the cost that will be accumulated by running the system from this point following an optimal policy. The idea behind it is to find a policy which will minimize the cost function as time trends toward infinity.

We can then formulate the Hamilton-Jacobi-Bellman equation for this cost function:

$$
\min_{\mathbf{u}} \left( \mathbf{x}^T Q \mathbf{x} + \mathbf{u}^T R \mathbf{u} + \frac{\partial J^*}{\partial \mathbf{x}} \left( A \mathbf{x} + B \mathbf{u} \right) \right) = 0, \; \; \forall \mathbf{x}
$$

The optimal cost-to-go function is quadratic, which means we can write it in the following shape:

$$
J^* (\mathbf{x})= \mathbf{x}^T S \mathbf{x}
$$

Then we can rewrite the gradient of the cost-to-go function as:

$$
\frac{\partial J^*}{\partial \mathbf{x}} = 2 \mathbf{x}^T S
$$

Now the derivative of the cost HJB equation with respect to $\mathbf{u}$ is equalized it to 0 to find the minimum:

$$
2 \mathbf{u}^T R + 2 \mathbf{x}^TSB = 0
$$

From the last equation, we isolate the optimal control law:

$$
\mathbf{u}^* = \pi^*(\mathbf{x}) = - R^{-1} B^T S \mathbf{x} = -\mathbf{K} \mathbf{x}
$$

Since we know this is an optimal policy, it satisfies the HJB equation. We insert it back and simplify:

$$
\mathbf{x}^T \left( Q - SBR^{-1}B^TS + 2SA\right) \mathbf{x} = 0
$$

Which can then be rearranged:

$$
\mathbf{x}^T \left( Q - SBR^{-1}B^TS + SA + A^TS \right)\mathbf{x} = 0
$$

Since the previous equality holds for all $\mathbf{x}$, the term in parenthesis must be a null matrix:

$$
A^TS + SA - SBR^{-1}B^TS + Q = \mathbf{0}
$$

This is a version of the Algebraic Riccati Equation. After solving it, we get $S$ and then we can derive the optimal control law for setpoint control:

$$
\mathbf{u}^* = -R^{-1}B^TS \mathbf{x} = -\mathbf{K} \mathbf{x}
$$

### Linearization

The previous procedure of deriving the optimal gain for our controller assumes that the system is linear. To apply this approach to nonlinear systems, we must first linearize it. From the dynamics $\dot{\mathbf{x}} = A\mathbf{x} + B\mathbf{u}$, we may linearize the dynamics of our system about a point using the a first order Taylor series:

$$
\dot{\mathbf{x}} \approx A_{lin} \mathbf{x} + B_{lin} {\mathbf{u}}
$$

where

$$
A_{lin} = \frac{\partial A}{\partial \mathbf{x}} \bigg|_{\mathbf{x} = \mathbf{x_{SP}}} \; \; \; \; \; \; B_{lin} = \frac{\partial B}{\partial \mathbf{x}} \bigg|_{\mathbf{x} = \mathbf{x_{SP}}}
$$

The setpoint which we will be operating around is $\mathbf{x_{SP}} = \left[ \begin{array}{cccc} \pi & 0 & 0 & 0 \end{array} \right]$. The system dynamics will be decently approximated in the neighborhood of the setpoint. The further away from the setpoint, the less accurate the approximation becomes.

This linearization is defined in the following cell:

In [ ]:
def symbolic_linear_matrices(self):
    """
    symbolic A- and B-matrix of the linearized dynamics (xd = Ax+Bu)
    """
    Alin = diff_to_matrix(smp.diff(self.f, self.x))
    Alin = sub_symbols(Alin, self.x, self.x0)
    Alin = sub_symbols(Alin, self.u, self.u0)

    Blin = diff_to_matrix(smp.diff(self.f, self.u)).T
    Blin = sub_symbols(Blin, self.x, self.x0)
    Blin = sub_symbols(Blin, self.u, self.u0)

    return Alin, Blin.T

SymbolicDoublePendulum.symbolic_linear_matrices = symbolic_linear_matrices

## 2. Implementation

We now create our LQR controller. In the next cell, the cost matrices $Q$ and $R$ are defined. We also add a low-pass filter to clean the the readings of the encoders. This cell also prints out the controller gain $\mathbf{K}$.

In [ ]:
x0 = [3.09216976, 0.26000977, -0.0349345, 0.02595628]

if robot == "acrobot":

    Q = np.diag([0.0, 0.0, 0.0, 0.0])
    R = np.diag([0.0, 0.0])

elif robot == "pendubot":

    Q = np.diag([0.0, 0.0, 0.0, 0.0])
    R = np.diag([0.0, 0.0])
    
goal = [np.pi, 0, 0, 0]
controller = LQRController(model_pars=mpar_con)
controller.set_goal(goal)
controller.set_cost_matrices(Q=Q, R=R)
controller.set_parameters(failure_value=0, cost_to_go_cut=10000)

if friction_compensation:
    controller.set_friction_compensation(damping=[0.004,0.004], coulomb_fric=[0.00305,0.0007777])

# Filter args
lowpass_alpha=[1.0,1.0,0.9,0.9]
filter_velocity_cut=0.1
#controller_real.set_filter(filter1) # Uncomment to add filter to the controller

controller.init()
print("Controller gain K:", getattr(controller, "K", "Not set"))

### Think-Pair-Share

Run the next cell to simulate the controller from an initial state close to the setpoint. If the LQR is unable to hold the position, try adjusing $Q$ and $R$ matrices, running the previous cell again and then run the simulation again.

In [ ]:
T, X, U, anim = sim.simulate_and_animate(
    t0=0.0,
    x0=x0,
    tf=tf,
    dt=dt,
    controller=controller,
    integrator=integrator,
    save_video=False,
    anim_dt=dt
)
html = HTML(anim.to_jshtml())
display(html)
plt.close()

plot_timeseries(
    T,
    X,
    U,
    pos_y_lines=[0, np.pi],
    tau_y_lines=[-torque_limit[active_act], torque_limit[active_act]],
)

Now we will try the controller on the real system. The first thing we will do is create a new controller with a low-pass filter. This will allow us to smooth out high-frequency changes in the state of the robot, avoiding overreaction to quick fluctuations.

For the experiment, run the next cell, balance the pendulum on its upright position and then press enter (in that order).

In [ ]:
# LQR test
a=robot
print(a)
T, X, U, U_des = sim.run_experiment(
    tf=tf,
    dt=dt,
    controller=controller,
    experiment_type="DoublePendulum",
    motors = motors # [shoulder, elbow]
    # actuated_joint="Both"
)
plot_timeseries(
    T,
    X,
    U,
    T_des=T,
    U_des=U_des,
    pos_y_lines=[0, np.pi],
    tau_y_lines=[-torque_limit[active_act], torque_limit[active_act]],
)

It is very likely that the first experiment failed. Often, the simulation and the real system have slight differences that make it necessary to tune the $Q$ and $R$ matrices for each separately. If the experiment failed, try modifying the cost matrices and running the experiment again until the controller can stabilize the system.
When the pendulum is able to stay upright, try to disturb it and see if it returns to the goal position.

## 3. Region of Attraction

As we saw, this controller is generally only able to stabilize the double pendulum if the initial state is close to the setpoint. We can approximate a set of initial states that will result in the system being stabilized. This set receives the name **Region of Attraction (RoA)**

LaSalle's invariance principle states that if, for a system with continuous dynamics defined by $\dot{x} = f(x)$, we can produce a scalar function $V(x)$ with continuous derivatives for which we have:

- $V(\mathbf{x})>0 \;\;\; \forall \mathbf{x} \in \mathcal{G}, \mathbf{x} \neq \mathbf{0}$ and $V(\mathbf{0}) = 0$
- $\dot{V}(\mathbf{x}) \leq 0 \;\;\; \forall \mathbf{x} \in \mathcal{G}$

then, the origin is locally asymptotically stable and set $\mathcal{G}$ is contained within its region of attraction.

We can exploit this information to narrow down a set which is entirely within the region of attraction of the origin $x = \left[ \begin{matrix} \pi & 0 & 0 & 0\end{matrix} \right]^T$. A simple way of doing so is to pick a shape for set $\mathcal{G}$ and scale it so that it is within the region of attraction. 

We will use an ellipsoid centered on the origin and defined by:

$$
V(\mathbf{x}) \leq \rho
$$

For our $V(\mathbf{x})$ function we will use the optimal cost-to-go. Explicitly, this means that $V(\mathbf{x}) = \mathbf{x}^T S \mathbf{x}$.

To find the value of $\rho$ that provides the largest region of attraction guaranteeing stability, we can follow the procedure:

- Set an initial value of $\rho$ which we know is unrealistically large.
- Start sampling points from inside the ellipsoid. If at that point the conditions outlined in LaSalle's invariance principle hold, that point is part of the region of attraction. If LaSalle's principle of invariance does not hold for that point, we reduce the size of the ellipsoid until the point is no longer in it.
- We repeat the previous step until we converge to the largest ellipsoid which only contains points in the region of attraction.

The next cell defines a function which produces an ellipsoid around the origin with ceritified stability. 

In [ ]:
def estimate_roa_najafi(plant, controller, goal, S, n):
    rho = 10.0
    for i in range(n):
        # sample initial state from sublevel set
        # check if it fullfills Lyapunov conditions
        x_bar = sampleFromEllipsoid(S, rho)
        x = goal + x_bar
        
        tau = controller.get_control_output(x)

        xdot = plant.rhs(0, x, tau)

        V = quadForm(S, x_bar)

        Vdot = 2 * np.dot(x_bar, np.dot(S, xdot))

        if V < rho and Vdot > 0.0:
            # if one of the lyapunov conditions is not satisfied
            rho = V

    return rho

Run the next cell to identify the size of the region of attraction of the controller and plot sections of the RoA around the origin.

In [ ]:
rho = estimate_roa_najafi(plant = plant,
                            controller = controller,
                            goal = np.array([np.pi, 0, 0, 0]),
                            S = np.asarray(controller.S),
                            n = 10000)
print("rho: ", rho)
plotEllipseSections([np.pi, 0, 0, 0], rho, np.asarray(controller.S))

### Think-Pair-Share

Do the sections of the region of attraction reflect the real behaviour of the controller?
If the RoA seems exceedingly small, it means that the system is not stable around the origin. How can we use the estimation of the RoA to improve the stability of our system?

## 4. Maximizing the RoA with CMA-ES

As you have been able to see, tuning $Q$ and $R$ for this system can be very tricky. However, one set of tools that can be exploited to improve the stability of the system are optimization algorithms like **CMA-ES** (Covariance matrix adaptation evolution strategies). This optimization algorithm does not require derivatives, which makes it ideal for problems in which the relation between the cost matrices and the output of the controller are hard to present using differentiable expressions. The way CMA-ES works for our problem is by trying different combinations of weights for the $Q$ and $R$ cost matrices. The objective function tries to maximize $\rho$. 

CMA-ES operates by sampling candidate solutions and evaluating them to update a covariance matrix that guides the following sampling steps.

### Think-Pair-Share

Run the following cell with different initial guesses (*init_pars*). Be sure to increase *maxfevals* for more refined results. You may also increase the number of cores you are using for this optimization to speed it up. By default, the optimization uses the model parameters defined at the beginning of the notebook.

In [ ]:
filename = "roa_opt"
c_par = roa_lqr_opt(
    model_par=mpar_con,
    goal=goal,
    init_pars=[1.0, 1.0, 1.0, 1.0, 1.0],
    par_prefactors=[20.0, 20.0, 10.0, 10.0, 10.0],
    bounds=[[0, 1], [0, 1], [0, 1], [0, 1], [0, 1]],
    maxfevals=100, #needs to be higher for proper optimization e.g. =1000
    sigma0=0.4,
    roa_backend="najafi",
    najafi_evals=50,
    robot=robot,
    save_dir="data/"+filename,
    plots=True,
    num_proc=5, # more cores are needed for reasonable runtime, be sure to leave a few for yourself.
    popsize_factor=4,
)

Run the following cell to plot the sections of the resulting Region of Attraction

In [ ]:
rho_new = np.loadtxt('data/'+filename+'/rho')
S_new = np.loadtxt('data/'+filename+'/Smatrix')

plotEllipseSections(goal, rho_new, S_new)

Use CMA-ES to maximize the RoA of your controller and substitute the results of the optimization into its $Q$ and $R$ matrices.

### Think-Pair-Share

Until now, we have been working with pendubot, which is arguably the easier version of this problem. Try to optimize the RoA of a controller for **acrobot**. You may allow a bit of torque from the unpowered motor if you dont succeed in stabilizing the real robot in acrobot mode.